# **파이썬으로 구글스프레드시트 다루기: `pygsheets` 패키지**

아마도 상당히 많은 사람들이 업무 혹은 개인 용도로 구글스프레드시트를 사용하고 있을 것이다.   
구글스프레드시트는 쉽게 말해 구글 아이디만 있다면 무료로 온라인 상에서 MS Office의 Excel의 기능을 상당 부분 사용할 수 있는 매우 편리한 애플리케이션으로, 특히 자동화와 협업에 강점을 가지고 있는 툴이라고 할 수 있다.   
훌륭한 편의성을 갖추고 있기 때문에 구글스프레드시트를 작은 규모의 DB로 사용하기에도 무리가 없고, pygsheets 같은 패키지를 이용하면 파이썬으로 구글스프레드시트를 컨트롤 할 수 있어 꽤나 유용하다.   
그렇다면 pygsheets를 사용해서 파이썬으로 구글스프레드시트의 데이터를 다루는 방법에 대해 알아보자.  
여기서 커버되지 않은 더 자세한 내용은 언제나 그렇듯 공식문서 참조가 필요하다. (공식문서: https://pygsheets.readthedocs.io/en/stable/)

## **1. Install Package 패키지 설치하기**
운영체제에 따라 아래와 같이 패키지를 설치한다.
- mac
    - `$ pip install pygsheets oauth2client` (구글 API 인증을 위한 oauth2client도 함께 설치)
- windows
    - `$ conda install -c marta-sd pygsheets`
    - 아래와 같은 에러가 발생하면 pip로 설치
        - 에러 메세지: `TypeError: parse() got an unexpected keyword argument 'transport_encoding'`
        - pip 설치 명령어
            - `$ conda install pip`
            - `$ pip install pygsheets`

## **2. 구글 API 셋업 & 인증**
파이썬으로 구글 드라이브와 구글스프레드시트에 접근하기 위해서는 아래와 같이 구글 API 셋업과 인증이 필요하다.

### 2.1 OAuth
- 2006년에 IETF(Internet Engineering Task Force) OAuth Working Group에서 개발되고 유지보수 되는 인증을 위한 표준 프로토콜
- 공식 문서: https://oauth.net/2/
- https://confluence.atlassian.com/bitbucket/oauth-endpoint-300220600.html

### 2.2 API 인증(Authorizing) 방법
- 참고 자료: http://pygsheets.readthedocs.io/en/latest/authorizing.html
- 설정 방법
    1. 구글 개발자 콘솔에 접속 - https://console.developers.google.com
    2. 왼쪽 상단의 프로젝트 만들기로 프로젝트 생성
    ![](image/27_pygsheets/001.png)
    3. 좌측의 `라이브러리` 탭에서 Google Sheets API, Google Drive API를 추가 (`사용 설정` 클릭)
    4. 좌측의 `사용자 인증 정보`탭으로 이동 → OAuth 동의 화면 구성하기 → 정보 입력 후 저장 (애플리케이션 이름만 넣어도 됨)
    7. `사용자 인증 정보`탭 → 사용자 인증정보 만들기 → OAuth 클라이언트 ID 
    9. 기타 선택 → 이름 설정후 확인 클릭
    10. OAuth 클라이언트 생성됨 → OAuth 2.0 클라이언트 ID 항목에서 우측 다운로드 버튼 클릭 (json 확장자의 키 파일 다운로드)
    ![](image/27_pygsheets/002.png)
    11. 다운로드 받은 파일을 노트북파일(.ipynb)과 같은 디렉토리에 위치하도록 파일을 이동 및 파일이름을 "client_secret.json"으로 변경 </br>(각자 사용하기 편리한 경로와 파일명으로 설정)

In [1]:
import pandas as pd
import pygsheets

아래 코드를 실행하면 auth 인증창이 실행되고 로그인 후에 허용을 하면 access_token을 받와서 입력 (처음 1회만)

In [7]:
gc = pygsheets.authorize(outh_file='./Study_Python/client_secret.json')
gc

- windows에서 아래의 에러가 발생하면 `no_cache = True` 옵션을 설정
    - 에러메세지: `FileNotFoundError: [Errno 2] No such file or directory`

In [1]:
# gc = pygsheets.authorize(outh_file='client_secret.json', no_cache = True)

## **3. Handling Sheets**

먼저 시트를 열고, 복사하고, 선택하고, 지우는 방법부터 다뤄보자.  
다음과 같은 테스트용 구글스프레트시트 `people`(파일명)을 만들어 pygsheets 실습을 진행해보겠다.
![](image/27_pygsheets/004.png)

### 3.1 Open file/sheet
- 구글 드라이브 내의 스프레드시트 파일명으로 오픈
- 파일 접근
    - `open` 함수 이용
    - 파라미터: 구글 드라이브 내의 스프레드시트 파일명
- 시트 접근
    - 시트 오브젝트에 대해 `.sheet1` 으로 첫번째 시트를 가져올수 있음
    - 처음에 항상 sheet1을 가져오고 다른 시트는 selecting 기능으로 선택함

In [8]:
file_nm = "people"

In [9]:
sh = gc.open(file_nm) # 파일 열기 (sh : 파일에 대한 객체)
sheet1 = sh.sheet1    # 시트 접근 (sheet1 : 첫번째 시트에 대한 객체)

In [10]:
sheet1

<Worksheet '시트1' index:0>

### 3.2 Create Sheet (시트 생성)
현재 시트가 하나 뿐인 people 파일에 새로운 시트 "new_sheet"를 추가할 수 있다.
- `add_worksheet` 함수 이용
- 파라미터: 생성할 시트이름, 행과 열의 크기

In [24]:
sheet2 = sh.add_worksheet("new_sheet", rows=20, cols=5)
sheet2

<Worksheet 'new_sheet' index:1>

`new_sheet` 시트가 생겼다!  </br>
![](image/27_pygsheets/005.png)

### 3.3 Copy Sheet (시트 복사)
기존의 시트를 복사해 새로운 시트를 만들 수 있다.
- `add_worksheet` 함수 이용
- 파라미터: `src_worksheet`=복사할 시트 객체

In [34]:
sheet3 = sh.add_worksheet("copied", src_worksheet=sheet1)
sheet3

<Worksheet 'copied' index:2>

`copied` 시트가 생겼다!  </br>
![](image/27_pygsheets/006.png)

### 3.4 Select Sheet (시트 선택)
하나의 파일에 여러 개의 시트가 존재할 때 시트의 리스트를 확인하고, 원하는 시트를 선택하는 방법을 알아보자.
- 전체 시트의 객체에서 원하는 시트 객체를 선택
- 제목과 순서에 대한 값으로 시트를 선택 가능

In [35]:
# 모든 시트 리스트로 가져오기
sheet_list = sh.worksheets()
sheet_list

[<Worksheet '시트1' index:0>,
 <Worksheet 'new_sheet' index:1>,
 <Worksheet 'copied' index:2>]

In [36]:
# 시트 제목으로 가져오기
new_sheet = sh.worksheet_by_title("new_sheet")
print(new_sheet)

<Worksheet 'new_sheet' index:1>


In [37]:
sheet2 == new_sheet

True

In [38]:
# index로 시트 가져오기
sheet0 = sh.worksheet("index", 0)
print(sheet0)

<Worksheet '시트1' index:0>


In [39]:
sheet0 == sheet1

True

In [40]:
# offset으로 가져오기
sheet0 = sh[0]
print(sheet0)

<Worksheet '시트1' index:0>


In [41]:
sheet0 == sheet1

True

### 3.5 시트 제목

In [42]:
sh[1].title = "NewSheet"

In [43]:
sh[1].title

'NewSheet'

### 3.6 시트 삭제

In [44]:
# 시트 내용 모두 삭제하기
sh[2].clear()

In [46]:
# 시트 삭제하기
sh.del_worksheet(sh[2])
# sh.del_worksheet(sheet3)

## **4. Handling Data**
이제는 좀 더 구체적으로, 시트에 들어있는 데이터를 실제로 다루는 방법을 알아보자.  
파이썬으로 구글스프레드시트의 데이터를 읽어오고, 수정하고, 셀 서식도 지정할 수 있다.

### 4.1 데이터 가져오기

#### 1) 전체 데이터 가져오기
- `get_all_records()`
- `get_all_values()`

In [48]:
# 모든 records(row) 불러오기 - 딕셔너리 타입: 첫 행이 key가 됨
sheet1.get_all_records()

[{'name': 'Paul', 'email': 'paul@naver.com', 'phone': '010-1111-1111'},
 {'name': 'Jennie', 'email': 'jennie@daum.net', 'phone': '010-1234-5678'},
 {'name': 'Chloe', 'email': 'chloe@gmail.com', 'phone': '010-9999-9999'}]

In [52]:
# sheet의 rows와 columns 수 설정
sh[0].rows = 10
sh[0].cols = 3

In [54]:
# 모든 데이터 행렬로 가져오기 (리스트 타입) - 설정된 rows, cols 수 전체에 대해 가져옴
all_data_sheet1 = sheet1.get_all_values(returnas='matrix')
all_data_sheet1

[['name', 'email', 'phone'],
 ['Paul', 'paul@naver.com', '010-1111-1111'],
 ['Jennie', 'jennie@daum.net', '010-1234-5678'],
 ['Chloe', 'chloe@gmail.com', '010-9999-9999'],
 ['', '', ''],
 ['', '', ''],
 ['', '', ''],
 ['', '', ''],
 ['', '', ''],
 ['', '', '']]

#### 2) 특정 범위/위치의 데이터 가져오기
- `get_values()`

In [55]:
# 위치를 지정하여 행렬 형태로 데이터 가져오기
some_data_sheet1 = sheet1.get_values(start=(1,1), end=(4,3), returnas='matrix')
some_data_sheet1

[['name', 'email', 'phone'],
 ['Paul', 'paul@naver.com', '010-1111-1111'],
 ['Jennie', 'jennie@daum.net', '010-1234-5678'],
 ['Chloe', 'chloe@gmail.com', '010-9999-9999']]

In [56]:
# A1에서 C4의 셀 리스트 가져옴
cell_list = sheet1.get_values('A1','C4', returnas='cells')
cell_list

[[<Cell A1 'name'>, <Cell B1 'email'>, <Cell C1 'phone'>],
 [<Cell A2 'Paul'>, <Cell B2 'paul@naver.com'>, <Cell C2 '010-1111-1111'>],
 [<Cell A3 'Jennie'>, <Cell B3 'jennie@daum.net'>, <Cell C3 '010-1234-5678'>],
 [<Cell A4 'Chloe'>, <Cell B4 'chloe@gmail.com'>, <Cell C4 '010-9999-9999'>]]

- `range(crange, returnas='cells')`: returns a list of cell objects from specified range

In [57]:
# A1에서 C4의 셀 리스트 가져옴
cell_list = sheet1.range('A1:C4')
cell_list

[[<Cell A1 'name'>, <Cell B1 'email'>, <Cell C1 'phone'>],
 [<Cell A2 'Paul'>, <Cell B2 'paul@naver.com'>, <Cell C2 '010-1111-1111'>],
 [<Cell A3 'Jennie'>, <Cell B3 'jennie@daum.net'>, <Cell C3 '010-1234-5678'>],
 [<Cell A4 'Chloe'>, <Cell B4 'chloe@gmail.com'>, <Cell C4 '010-9999-9999'>]]

In [58]:
cell_list[2][1].value

'jennie@daum.net'

- `get_row(row, returnas='matrix', include_tailing_empty=True, **kwargs)`

In [62]:
# 두번째 row의 셀 리스트 가져옴
cell_list = sheet1.get_row(2, returnas='cells')
cell_list

[<Cell A2 'Paul'>, <Cell B2 'paul@naver.com'>, <Cell C2 '010-1111-1111'>]

#### 3) 특정 셀 데이터 가져오기

In [63]:
# "시트[행][열]"로 특정 셀의 데이터를 가져올 수 있음 (행은 1부터, 열은 0부터??)
value = sheet1[2][1]
value

'paul@naver.com'

In [64]:
value11 = sheet1[1][0]
value11

'name'

In [65]:
# 특정 셀의 객체 가져오기
b2 = sheet1.cell('B2')

# 셀 값 확인
print(b2.value)

paul@naver.com


In [66]:
b2.col

2

In [67]:
# 3을 b2 객체의 컬럼으로 할당
b2.col = 3

# 셀 값 확인
print(b2.value)

010-1111-1111


In [69]:
# b2에 해당하는 위치의 데이터를 "010-2222-2222"로 바꿈
b2.value = "010-2222-2222"
b2.value

'010-2222-2222'

![](image/27_pygsheets/007.png)

### 4.2 문자열을 포함한 cell 찾기

In [70]:
# 문자열 찾기
cell_list = sh[0].find("@gmail.com")
print(cell_list)

[<Cell B4 'chloe@gmail.com'>]


In [71]:
cell_list = sh[0].find("jennie")
print(cell_list)

[<Cell A3 'Jennie'>, <Cell B3 'jennie@daum.net'>]


### 4.3 Update & Insert

#### 1) 특정 범위에 데이터 업데이트
- `update_values(crange=None, values=None, cell_list=None, extend=False, majordim='ROWS', parse=None)`

'NewSheet' 시트의 A1:C4에 some_data_sheet1 데이터를 업데이트하기

In [72]:
sh[1]

<Worksheet 'NewSheet' index:1>

In [73]:
some_data_sheet1

[['name', 'email', 'phone'],
 ['Paul', 'paul@naver.com', '010-1111-1111'],
 ['Jennie', 'jennie@daum.net', '010-1234-5678'],
 ['Chloe', 'chloe@gmail.com', '010-9999-9999']]

In [81]:
sh[1].update_values(crange='A1:C4', values=some_data_sheet1)

In [82]:
# 시트의 열과 행을 재설정해줌
sh[1].rows = 6
sh[1].cols = 3

![](image/27_pygsheets/009.png)

In [83]:
# sh[1] 시트의 모든 데이터 가져오기
all_data_sheet2 = sh[1].get_all_values()
all_data_sheet2

[['name', 'email', 'phone'],
 ['Paul', 'paul@naver.com', '010-1111-1111'],
 ['Jennie', 'jennie@daum.net', '010-1234-5678'],
 ['Chloe', 'chloe@gmail.com', '010-9999-9999'],
 ['', '', ''],
 ['', '', '']]

#### 2) 특정 셀 데이터 업데이트
- syntax: `update_value(addr, val, parse=None)`

In [84]:
sh[1].update_value('C2', '010-3333-3333')

#### 2) insert & append

In [85]:
# 4번째줄 아래로 row insert (행을 삽입하면서 데이터를 넣어줄 수 있음)
sh[1].insert_rows(row=4, number=1, values=all_data_sheet2)

In [86]:
# 반복문을 통해 한줄씩 읽어 올수 있음
for row in sh[1]:
    print(row)

['name', 'email', 'phone']
['Paul', 'paul@naver.com', '010-3333-3333']
['Jennie', 'jennie@daum.net', '010-1234-5678']
['Chloe', 'chloe@gmail.com', '010-9999-9999']
['name', 'email', 'phone']
['Paul', 'paul@naver.com', '010-1111-1111']
['Jennie', 'jennie@daum.net', '010-1234-5678']
['Chloe', 'chloe@gmail.com', '010-9999-9999']


In [87]:
# 시트의 마지막 데이터를 찾아 마지막 데이터의 아래에 데이터를 추가
sh[1].append_table(values=["Ryan","ryan@gmail.com"])

In [88]:
sh[1].rows

11

![](image/27_pygsheets/010.png)

### 4.4 셀 서식 지정
셀에 메모를 추가하거나 셀 배경 색상, 폰트 크기 등의 텍스트 포맷 설정 등 셀 서식 지정도 가능하다.

In [96]:
cell = sheet1.cell('C2')

# 노트 추가
cell.note = "this is phone number."

# 셀 배경 색상 변경 (Red, Green, Blue, Alpha)
cell.color = (1.0, 1.0, 0.0, 1.0) 

# 텍스트 포멧 변경
cell.set_text_format('fontSize', 12)
cell.set_text_format('bold', True)

<Cell C2 '010-2222-2222'>

![](image/27_pygsheets/011.png)

## **5. Pandas DataFrame과의 호환**

사실 구글스프레드시트에서 데이터를 불러와 실제로 분석 작업 등을 편하게 하기 위해서는 Pandas와의 호환이 가장 중요하다고 할 수 있다. 구글스프레드시트에서 DataFrame으로 가져올 수만 있다면 평소에 pandas로 하던 데이터 관련작업을 그대로 사용하면 되니까.  
그렇다면 구글스프레드시트를 DataFrame으로 가져오는 방법, 반대로 내가 가진 DataFrame을 구글스프레드시트로 보내는 방법을 알아볼까.

### 5.1 Sheet → DataFrame
- `get_as_df()` 함수 이용

In [97]:
sheet1

<Worksheet '시트1' index:0>

In [98]:
df = sheet1.get_as_df()
df

,name,email,phone
0,Paul,paul@naver.com,010-2222-2222
1,Jennie,jennie@daum.net,010-1234-5678
2,Chloe,chloe@gmail.com,010-9999-9999


In [99]:
# csv 파일로 내보내기
df.to_csv("email.csv", index=False)

### 5.2 DataFrame → sheet
- `set_dataframe(DataFrame, cell_start)` 함수 이용

#### 1) csv 파일을 불러와서 구글스프레드시트로 보내기

In [100]:
# csv 파일 읽어오기
df = pd.read_csv("email.csv")
df

,name,email,phone
0,Paul,paul@naver.com,010-2222-2222
1,Jennie,jennie@daum.net,010-1234-5678
2,Chloe,chloe@gmail.com,010-9999-9999


In [101]:
# 새로운 시트 만들기
email_sheet = sh.add_worksheet("email")

In [102]:
email_sheet.set_dataframe(df, 'A1', copy_index=False) 

![](image/27_pygsheets/012.png)

#### 2) seaborn에서 iris 데이터를 가져와서 구글 스프레드시트에 넣기

In [103]:
import seaborn as sns

iris = sns.load_dataset("iris")
iris.tail()

,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [104]:
iris_sheet = sh.add_worksheet("iris")  # 새로운 시트 만들기

In [105]:
iris_sheet.set_dataframe(iris, 'A1', copy_index=True) # (df, cell_start)

![](image/27_pygsheets/013.png)

## **6. Share**
구글스프레드시트의 공유 권한을 추가하거나 삭제하는 것도 아래와 같이 가능하다.

### 6.1 공유 권한 추가

In [169]:
# role: 'owner','writer','commenter','reader'
sh.share("hyeshinoh+h@gmail.com", role='reader')

### 6.2 공유 권한 삭제

In [171]:
sh.remove_permission("hyeshinoh+h@gmail.com")